# Target Description Language

In [1]:
from keckODL.target import Target, TargetList

In [2]:
# Example: fully specify target.  Coordinates in decimal degrees.
t1 = Target(name='M42', RA=82.5, Dec=-5, frame='icrs',
            PMRA=10, PMDec=-10, epoch=1975.5,
            rotmode='PA', PA=90,
            objecttype='science',
            RAOffset=8.4, DecOffset=-4.5,
            acquisition='guider: offset',
            mag={'V': 4.3, 'K': 6.7},
            comment='Star formation is cool',
            wrap='shortest',
           )
t1

M42              05 30 29.62 -05 07 22.44 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 wrap=shortest vmag=4.30 # Vmag=4.30 Kmag=6.70 Star formation is cool

In [3]:
# Example: fully specify target.  Coordinates in hmsdms.
t2 = Target(name='M31', RA='00:42:44.33', Dec='41:16:07.50',
            rotmode='PA', PA=32,
            RAOffset=12.1, DecOffset=-1.9,
            objecttype='science',
            acquisition='guider: offset',
            mag={'V': 2.3, 'K': 8.7},
            comment='Galaxies are boring',
            wrap='north',
           )
t2

M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 wrap=north vmag=2.30 # Vmag=2.30 Kmag=8.70 Galaxies are boring

In [4]:
# Example: Resolve target by name.
t3 = Target('NGC1333',
            rotmode='PA', PA=22.5,
            objecttype='science',
            acquisition='blind',
            comment='Low mass star formation is the coolest')
t3

NGC1333          03 29 11.28 +31 18 36.00 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest

In [5]:
# Example: Resolve target by name, set vertical angle mode, set obstype
t4 = Target('Feige110',
            objecttype='flux standard',
            rotmode='vertical',
            acquisition='guider: bright',
            comment='I guess we need a standard')
t4

/Users/joshw/git/KeckODL/keckODL/target.py:257: TargetWarning: No PA given, assuming 0 deg
  warn('No PA given, assuming 0 deg', category=TargetWarning)


Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

In [6]:
# Build a target list, write as TDL TAML file and as starlist
tl = TargetList([t1, t2, t3, t4])
tl.write('targets.txt')
tl.write_starlist('starlist.txt')
tl

M42              05 30 29.62 -05 07 22.44 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 wrap=shortest vmag=4.30 # Vmag=4.30 Kmag=6.70 Star formation is cool
M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 wrap=north vmag=2.30 # Vmag=2.30 Kmag=8.70 Galaxies are boring
NGC1333          03 29 11.28 +31 18 36.00 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest
Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

In [7]:
# Read in a target list from the TDL YAML file
newtl = TargetList().read('targets.txt')
newtl

M42              05 30 29.62 -05 07 22.42 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 vmag=4.30 # Kmag=6.70 Vmag=4.30 Star formation is cool
M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 vmag=2.30 # Kmag=8.70 Vmag=2.30 Galaxies are boring
NGC1333          03 29 11.28 +31 18 36.00 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest
Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

In [8]:
from astropy.time import Time
tl.set_obstime(Time(2045.11, format='decimalyear'))
tl

M42              05 30 45.44 -05 11 18.69 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 wrap=shortest vmag=4.30 # Vmag=4.30 Kmag=6.70 Star formation is cool
M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 wrap=north vmag=2.30 # Vmag=2.30 Kmag=8.70 Galaxies are boring
NGC1333          03 29 11.28 +31 18 36.00 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest
Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

# Observing Sequences

## KCWI Example

In [9]:
from keckODL.kcwi import KCWIblueDetectorConfig
from keckODL.kcwi import KCWIblueConfig
from keckODL.offset import Stare, StarSkyStar
from keckODL.sequence import SequenceElement, Sequence

In [10]:
kcwi_1800s = KCWIblueDetectorConfig(exptime=1800, binning='2x2')
kcwi_1800s.write('kcwi_1800s.txt')
print(kcwi_1800s)

KCWIblue 1800.0s (9)


In [11]:
med_slicer_4800 = KCWIblueConfig(slicer='medium', grating='BH3', cwave=4800)
med_slicer_4800.write('KCWI_medslicer_4800.txt')
print(med_slicer_4800)

medium BH3 4800 A


In [12]:
skyoffset = StarSkyStar(dx=30, dy=30)
print(skyoffset)

StarSkyStar (30 30)


In [13]:
skyoffset

Frame: SkyFrame (RAOFF DECOFF)
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
  +0.0|  +0.0|    +0.0|    star|  True
 +30.0| +30.0|    +0.0|     sky| False
  +0.0|  +0.0|    +0.0|    star|  True

In [14]:
skyoffset[1].execute()

'RAOFF'.write(30, rel2curr=f, rel2base=t)
'DECOFF'.write(30, rel2curr=f, rel2base=t)


In [15]:
myseq = Sequence([SequenceElement(pattern=skyoffset,
                                  detconfig=kcwi_1800s,
                                  instconfig=med_slicer_4800,
                                  repeat=3),
                  ])
print(myseq)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
StarSkyStar (30 30)|     3 |KCWIblue 1800.0s (9)         |medium BH3 4800 A                            


In [16]:
myseq.estimate_time()

Shutter Open Time: 16200 s (4.5 hrs)
Wall Clock Time: 16200 s (4.5 hrs)


In [17]:
med_slicer_4800.cals()

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
Stare              |     1 |KCWIblue 6.0s (9)            |medium BH3 4800 A arclamp=CONT calobj=MedBarsA
Stare              |     1 |KCWIblue 30.0s (9)           |medium BH3 4800 A arclamp=FEAR calobj=FlatA  
Stare              |     1 |KCWIblue 45.0s (9)           |medium BH3 4800 A arclamp=THAR calobj=FlatA  
Stare              |     6 |KCWIblue 6.0s (9)            |medium BH3 4800 A arclamp=CONT calobj=FlatA  
Stare              |     7 |KCWIblue 0.0s (9, Dark)      |medium BH3 4800 A                            
Stare              |     3 |KCWIblue 100.0s (9)          |medium BH3 4800 A domeflatlamp=True          

## MOSFIRE Example

In [18]:
from keckODL.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, Long2pos, ABBA
from keckODL.offset import Stare
from keckODL.sequence import SequenceElement, Sequence

In [19]:
# detector configs
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_180s.write('mosfire_180s.txt')
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
mosfire_120s.write('mosfire_120s.txt')
print(mosfire_180s)
print(mosfire_120s)

MOSFIRE 180.0s (MCDS16, 1 coadds)
MOSFIRE 120.0s (MCDS16, 1 coadds)


In [20]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

my_MOS_mask Y-spectroscopy
my_MOS_mask J-spectroscopy
my_MOS_mask H-spectroscopy
my_MOS_mask K-spectroscopy


In [21]:
abba = ABBA(offset=1.5)
print(abba)

ABBA (1.5)


In [22]:
abba

Frame: MOSFIRE Slit (INSTXOFF INSTXYOFF)
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
  +0.0|  +1.5|    +0.0|       A|  True
  +0.0|  -1.5|    +0.0|       B|  True
  +0.0|  -1.5|    +0.0|       B|  True
  +0.0|  +1.5|    +0.0|       A|  True

In [23]:
abba[0].execute()

'INSTXOFF'.write(0, rel2curr=f, rel2base=t)
'INSTXYOFF'.write(1.5, rel2curr=f, rel2base=t)


In [24]:
# sequence
myseq_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_Y, repeat=5),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_J, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_H, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_K, repeat=5),
                         ])
print(myseq_mosfire)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
ABBA (1.5)         |     5 |MOSFIRE 180.0s (MCDS16, 1 coadds)|my_MOS_mask Y-spectroscopy                   
ABBA (1.5)         |     8 |MOSFIRE 120.0s (MCDS16, 1 coadds)|my_MOS_mask J-spectroscopy                   
ABBA (1.5)         |     8 |MOSFIRE 120.0s (MCDS16, 1 coadds)|my_MOS_mask H-spectroscopy                   
ABBA (1.5)         |     5 |MOSFIRE 180.0s (MCDS16, 1 coadds)|my_MOS_mask K-spectroscopy                   


In [25]:
myseq_mosfire.estimate_time()

Shutter Open Time: 14880 s (4.1 hrs)
Wall Clock Time: 14880 s (4.1 hrs)


In [26]:
cals_I_need = mosfire_Y.cals()
cals_I_need.extend( mosfire_J.cals() )
cals_I_need.extend( mosfire_H.cals() )
cals_I_need.extend( mosfire_K.cals() )
print(cals_I_need)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|my_MOS_mask Y-spectroscopy domeflatlamp=True 
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|my_MOS_mask J-spectroscopy domeflatlamp=True 
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|my_MOS_mask H-spectroscopy domeflatlamp=True 
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|my_MOS_mask K-spectroscopy domeflatlamp=True 
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|my_MOS_mask K-spectroscopy domeflatlamp=False
Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds) |my_MOS_mask K-spectroscopy arclamp=Ne        
Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds) |my_MOS_mask K-spectroscopy arclamp=Ar        


In [27]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)


## MOSFIRE Example 2

In [28]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(singleobj_Y)
print(singleobj_J)
print(singleobj_H)
print(singleobj_K)

longslit_3x0.7 Y-spectroscopy
longslit_3x0.7 J-spectroscopy
long2pos H-spectroscopy
long2pos K-spectroscopy


In [29]:
long2pos = Long2pos()
print(long2pos)

long2pos


In [30]:
long2pos

Frame: MOSFIRE Detector (INSTXOFF INSTXYOFF)
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
 +45.0| -23.0|    +0.0|       A|  True
 +45.0|  -9.0|    +0.0|       B|  True
 -45.0|  +9.0|    +0.0|       A|  True
 -45.0| +23.0|    +0.0|       B|  True

In [35]:
long2pos[0].execute()

'INSTXOFF'.write(45, rel2curr=f, rel2base=t)
'INSTXYOFF'.write(-23, rel2curr=f, rel2base=t)


In [31]:
# sequence
myseq2_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                           SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                           SequenceElement(pattern=long2pos, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                           SequenceElement(pattern=long2pos, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                          ])
print(myseq2_mosfire)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
ABBA (1.5)         |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)|longslit_3x0.7 Y-spectroscopy                
ABBA (1.5)         |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)|longslit_3x0.7 J-spectroscopy                
long2pos           |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)|long2pos H-spectroscopy                      
long2pos           |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)|long2pos K-spectroscopy                      


In [32]:
myseq2_mosfire.estimate_time()

Shutter Open Time: 4800 s (1.3 hrs)
Wall Clock Time: 4800 s (1.3 hrs)


In [33]:
cals_I_need = singleobj_Y.cals()
cals_I_need.extend(singleobj_J.cals())
cals_I_need.extend(singleobj_H.cals())
cals_I_need.extend(singleobj_K.cals())
print(cals_I_need)

Pattern            |repeat |DetectorConfig               |InstrumentConfig                             
-------------------|-------|-----------------------------|---------------------------------------------
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|longslit_3x0.7 Y-spectroscopy domeflatlamp=True
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|longslit_3x0.7 J-spectroscopy domeflatlamp=True
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|long2pos H-spectroscopy domeflatlamp=True    
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|long2pos K-spectroscopy domeflatlamp=True    
Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)|long2pos K-spectroscopy domeflatlamp=False   
Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds) |long2pos K-spectroscopy arclamp=Ne           
Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds) |long2pos K-spectroscopy arclamp=Ar           


In [34]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)
